# Bokeh Line Chart

In [ ]:
# Import data
import pandas as pd
df_business = pd.read_csv("../data/cleaned_businessV2.csv")
df_review = pd.read_csv("../data/crosslisted_reviews.csv")

# Rename "stars", so we know where each column come from
df_business.rename(columns={'stars' : 'business_stars'}, inplace=True)
df_review.rename(columns={'stars' : 'review_stars'}, inplace=True)

# Join the two tables
df_joined = pd.merge(df_business, df_review, on="business_id", how="inner")
# Convert column types to string
df_joined = df_joined.convert_dtypes()
df_joined["review_stars"] = df_joined["review_stars"].astype(float)

# Check for NaN
#print(df_joined.isna().sum())

# Drop columns with null address and postal code. 
# Turnned out the ones without postal codes also did not have addresses 
df_joined.dropna(subset="address", inplace=True)
print(df_joined.isna().sum())


business_id        0
name               0
address            0
city               0
state              0
postal_code        0
latitude           0
longitude          0
business_stars     0
review_count       0
is_open            0
categories         0
hours_Monday       0
hours_Tuesday      0
hours_Wednesday    0
hours_Thursday     0
hours_Friday       0
hours_Saturday     0
hours_Sunday       0
review_id          0
review_stars       0
date               0
dtype: int64


In [26]:
df_joined["date"].head()

0    2014-05-26 01:09:53
1    2013-10-05 15:19:06
2    2013-10-25 01:34:57
3    2018-05-20 17:58:57
4    2013-10-25 02:31:35
Name: date, dtype: string

In [36]:
# Define kinds of restaurants we are interested in. May need to delete this later
# to allow the user to define this with UI
categories_of_interest = ['Chinese', 'Japanese', 'Italian', 'Polish', 'Scandinavian']

# Create new column containing a specific category of interest. 
# If not in interest, label the column value "Other"
df_joined['category_of_interest'] = "Other"
for item in categories_of_interest:
    df_joined.loc[df_joined['categories'].str.contains(item), 'category_of_interest'] = item

# Extract the year from the date time string.
df_joined["Year"] = df_joined["date"].apply(lambda x: x.split("-")[0])

df_grouped = df_joined.groupby(['category_of_interest', "Year"])["review_stars"].mean()
df_grouped.head(2)


category_of_interest  Year
Chinese               2005    4.000000
                      2006    4.058824
Name: review_stars, dtype: float64

In [35]:
df_unstacked = df_grouped.unstack("category_of_interest")
df_unstacked.head(2)

category_of_interest,Chinese,Italian,Japanese,Other,Polish,Scandinavian
Year,,,,,,
2005,4.000000,3.944444,4.500000,3.915493,NaN,NaN
2006,4.058824,4.024390,4.305556,3.937722,5.0,NaN


In [41]:
from bokeh.plotting import figure, show
from bokeh.palettes import Colorblind

# create a new plot with a title and axis labels
p = figure(title="Average Rating by Category", 
           x_axis_label='Year', 
           y_axis_label='Rating')

# Get a list of colorblind friendly colors of length = len(categories_of_interest) 
colors = Colorblind[len(categories_of_interest)]

print(df_unstacked.columns)

#for category in categories_of_interest:
#    # add a line renderer with legend and line thickness to the plot
#    p.line(df_unstacked["Year"], df_unstacked[category], legend_label="Temp.", line_width=2)

Index(['Chinese', 'Italian', 'Japanese', 'Other', 'Polish', 'Scandinavian'], dtype='object', name='category_of_interest')
